In [1]:
# def install_rapids():
#     import sys
#     !cp ../input/rapids/rapids.0.12.0 /opt/conda/envs/rapids.tar.gz
#     !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
#     sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
#     sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
#     sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
#     !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
# install_rapids()

In [4]:
!conda install -c rapidsai -c nvidia -c conda-forge -c defaults rapids=0.13 python=3.7

Solving environment: - ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt



In [3]:
!python --version

Python 3.7.6


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.metrics import f1_score, accuracy_score
from cuml.neighbors import KNeighborsClassifier, NearestNeighbors
import cuml; cuml.__version__

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy c-extensions failed.
- Try uninstalling and reinstalling numpy.
- If you have already done that, then:
  1. Check that you expected to use Python3.7 from "/opt/conda/bin/python",
     and that you have no directories in your PATH or PYTHONPATH that can
     interfere with the Python and numpy version "1.18.1" you're trying to use.
  2. If (1) looks fine, you can open a new issue at
     https://github.com/numpy/numpy/issues.  Please include details on:
     - how you installed Python
     - how you installed numpy
     - your operating system
     - whether or not you have multiple versions of Python installed
     - if you built from source, your compiler versions and ideally a build log

- If you're working with a numpy git repository, try `git clean -xdf`
  (removes all files not under version control) and rebuild numpy.

Note: this error has many possible causes, so please don't c

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    def __init__(self,  k_list, n_jobs=1, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.batch_size = 100_000
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list))
        self.NN.fit(X)        
        self.y_train = y
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
    def get_batches(self, batch_size, neighs_dist, neighs):
        n = len(neighs_dist)
        it = int(np.ceil(n/batch_size))

        batches = [(neighs_dist[batch_size*i:batch_size*(i+1)],
                    neighs[batch_size*i:batch_size*(i+1)]) for i in range(it)]
        return batches
        
    def predict(self, X):       
        neighs_dist, neighs = self.NN.kneighbors(X)
        batches = self.get_batches(self.batch_size, neighs_dist, neighs)
                
        test_feats = []
        if self.n_jobs == 1:
            for i, (neighs_dist, neighs) in enumerate(batches):
                print(f'Batch number {i}...')
                test_feats.append(self.get_features_for_batch(neighs_dist, neighs))            
#         with Pool(self.n_jobs) as p:
#             test_feats = p.map(self.get_features_for_batch, batches)                       
        return np.vstack(test_feats)
        
        
    def get_features_for_batch(self, neighs_dist, neighs):
        
        neighs_y = self.y_train[neighs.astype(int)] 
        return_list = []
        
        def bincount(x):
            return np.bincount(x, minlength=self.n_classes)
        
        print('1. Labels distribution')
        for k in self.k_list:
            feats = np.apply_along_axis(bincount, axis=1, arr=neighs_y[:,:k])
            feats = feats/ feats.sum(axis=1, keepdims=True)
            assert feats.shape == (neighs_dist.shape[0], self.n_classes)
            return_list += [feats]
            
        print('2. Mean')
        for k in self.k_list:
            feats = np.mean(neighs_y[:,:k], axis=1, keepdims=True)
            assert feats.shape == (neighs_dist.shape[0], 1)
            return_list += [feats]

        print('3. Quantilles')
        q = np.arange(0.0, 1.1, 0.1)
        for k in self.k_list:
            feats = np.quantile(neighs_y[:,:k], q=q, axis=1).T
            assert feats.shape == (neighs_dist.shape[0], 11)
            return_list += [feats]
        
        knn_feats = np.hstack(return_list)
        return knn_feats

In [4]:
%% time
X = np.random.randn(5_00_000, 10)
y = np.random.randint(10, size=(5_00_000))
nn_feats = NearestNeighborsFeats(k_list=[51, 101])
nn_feats.fit(X, y)
feats = nn_feats.predict(X)

(500000, 10) 101
Batch number 0...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 1...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 2...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 3...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 4...
1. Labels distribution
2. Mean
3. Quantilles


In [1]:
NUMBER_OF_KNN_FEATS = 44

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Dense, Add, Multiply, GRU, Bidirectional, LSTM
import pandas as pd
import numpy as np
import random
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import gc

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import warnings
import os
from sklearn.model_selection import cross_val_predict

In [6]:
# configurations and main hyperparammeters

GROUP_BATCH_SIZE = 4000
SEED = 321
SPLITS = 5

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [11]:
# read data
def read_data():
    train = pd.read_csv('/kaggle/input/data-without-drift/train_clean.csv', dtype={'time': np.float32, 'signal': np.float32, 'open_channels':np.int32})
    test  = pd.read_csv('/kaggle/input/data-without-drift/test_clean.csv', dtype={'time': np.float32, 'signal': np.float32})
    sub  = pd.read_csv('/kaggle/input/liverpool-ion-switching/sample_submission.csv', dtype={'time': np.float32})

    return train, test, sub



# create batches of 4000 observations
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df

# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test

# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:    
        df['signal_shift_pos_' + str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' + str(window)] = df.groupby('group')['signal'].shift(-1 * window).fillna(0)
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal'] ** 2
    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'group', 'open_channels', 'time']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

#prepare dataset for knn
def preproc_for_knn(df):
    feats = ['signal_shift_neg_3','signal_shift_neg_2','signal_shift_neg_1','signal',
            'signal_shift_pos_1','signal_shift_pos_2', 'signal_shift_pos_3']
    
    coefs = np.array([0.25, 0.5, 1.0, 4.0, 1.0, 0.5, 0.25])
    return df[feats] * coefs
    
#split knn dataset into hand-crafted groups 
def get_groups_for_knn(train, test):
    train_knn_group = np.full(-1, train.shape[0])
    
    x = [(0,500000),(1000000,1500000),(1500000,2000000),(2500000,3000000),(2000000,2500000)]
    for k in range(5): train_knn_group[x[k][0]:x[k][1]] = k
        
    test_knn_group = np.full(-1, test.shape[0])
    x = [[(0,100000),(300000,400000),(800000,900000),(1000000,2000000)],[(400000,500000)], 
         [(100000,200000),(900000,1000000)],[(200000,300000),(600000,700000)],[(500000,600000),(700000,800000)]]
    
    for k in range(5):
        for j in range(len(x[k])): test_knn_group[x[k][j][0]:x[k][j][1]] = k
    
    assert not -1 in train_knn_group
    assert not -1 in test_knn_group

    return train_knn_group, test_knn_group

def knn_feats_train_predict_by_groups(train, test, train_knn_group, test_knn_group):
    feats = np.zeros(test.shape[0], NUMBER_OF_KNN_FEATS)
    for group in range(5):
        model = NearestNeighborsFeats(k_list=[51, 101])
        model.fit(train[train_knn_group==group])
        feats[test_knn_group==group] = model.predict(test[test_knn_group==group])
        
    return feats
    
def run_cv_model_by_batch(train, test, splits, batch_col, feats, sample_submission):
    
    seed_everything(SEED)
    
    oof_ = np.zeros((len(train), 11)) # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
    preds_ = np.zeros((len(test), 11))
    
    target = ['open_channels']
    group = train['group']
    
    kf = GroupKFold(n_splits=5)
    cv = kf.split(train, train[target], group)    
    
    target = train.open_channels.values
    train = preproc_for_knn(train).values
    test = preproc_for_knn(test).values
    
    print(train.shape, test.shape)
    
    train_knn_group, test_knn_group = get_groups_for_knn(train, test)

    oof_knn_feats = np.zeros(train.shape[0], NUMBER_OF_KNN_FEATS)    
    
    for train_idx, val_idx in cv:
        oof_knn_feats[val_idx] = knn_feats_train_predict_by_groups(
            train[train_idx], train[val_idx], train_knn_group[train_idx], train_knn_group[val_idx])
        
    np.save('oof_knn_feats.npy', oof_knn_feats)
  

    test_knn_feats = knn_feats_train_predict_by_groups(train, test, train_knn_group, test_knn_group)
    np.save('test_knn_feats.npy', test_knn_feats)

    index_of_median_feature = -6
    oof_pred = oof_knn_feats[:, index_of_median_feature]
    
    test_pred = test_knn_feats[:, index_of_median_feature]
    f1_score_ = f1_score(oof_pred, target, average='macro') 
    
    print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
    sample_submission['open_channels'] = test_pred.astype(int)
    sample_submission.to_csv('submission_knn.csv', index=False, float_format='%.4f')
    
 
    
# this function run our entire program
def run_everything():
    
    print('Reading Data Started...')
    train, test, sample_submission = read_data()
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')
        
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size = GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size = GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')
        
   
    print(f'Training Wavenet model with {SPLITS} folds of GroupKFold Started...')
    run_cv_model_by_batch(train, test, SPLITS, 'group', features, sample_submission)
    print('Training completed...')
        
run_everything()

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Feature Engineering Completed...
Training Wavenet model with 5 folds of GroupKFold Started...
(5000000, 7) (2000000, 7)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/envs/rapids/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


(4000000, 7) 101
Batch number 0...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 1...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 2...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 3...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 4...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 5...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 6...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 7...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 8...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 9...
1. Labels distribution
2. Mean
3. Quantilles
(4000000, 7) 101
Batch number 0...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 1...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 2...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 3...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 4...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 5...
1. 

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  9.3min finished


(5000000, 7) 101
Batch number 0...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 1...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 2...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 3...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 4...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 5...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 6...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 7...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 8...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 9...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 10...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 11...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 12...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 13...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 14...
1. Labels distribution
2. Mean
3. Quantilles
Batch number 15...
1. Labels dist